This notebook contains an example of using `redbiom` through it's Python API to extract a subset of American Gut Project samples. These data are then loaded into QIIME 2 for a mini beta-diversity analysis using UniFrac. This assumes we're using a QIIME 2 2018.11 environment that additionally has `redbiom` 0.3.0 installed. The exact commands I ran to install it are:

```
$ conda install nltk
$ pip install https://github.com/biocore/redbiom/archive/0.3.0.zip
```

In [1]:
import redbiom.summarize
import redbiom.search
import redbiom.fetch
import qiime2
import pandas as pd
pd.options.display.max_colwidth = 1000

The first thing we're going to do is gather the `redbiom` contexts. A context is roughly a set of consistent technical parameters. For example, the specific sequenced gene, the variable region within the gene, the length of the read, and how the operational taxonomic units were assessed.

The reason `redbiom` partitions data into contexts is because these technical details can lead to massive technical bias. The intention is to facilitate comparing "apples" to "apples". 

The context summarization returns a pandas `DataFrame` so it should be pretty friendly to manipulate.

In [2]:
contexts = redbiom.summarize.contexts()

In [3]:
contexts.shape

(104, 4)

In [4]:
contexts.head()

,ContextName,SamplesWithData,FeaturesWithData,Description
0,Pick_closed-reference_OTUs-Greengenes-ls454-16S-v35-90nt-44feac,6067,7081,Qiita context
1,Pick_closed-reference_OTUs-Greengenes-flx-16S-v6-100nt-a243a1,107,4543,Qiita context
2,Pick_closed-reference_OTUs-Greengenes-iontorrent-16S-v3-150nt-bd7d4d,32,1008,Qiita context
3,Pick_closed-reference_OTUs-Greengenes-illumina-18S-v9-5c6506,206,309,Qiita context
4,Pick_closed-reference_OTUs-Greengenes-titanium-16S-v34-90nt-44feac,57,818,Qiita context


At the present time, determining the context to use is a bit manual and requires some strung munging. Additional development is needed.

Let's take a look at the larger contexts.

In [5]:
contexts.sort_values('SamplesWithData', ascending=False).head()

,ContextName,SamplesWithData,FeaturesWithData,Description
77,Pick_closed-reference_OTUs-Greengenes-illumina-16S-v4-100nt-a243a1,129596,74983,Qiita context
85,Pick_closed-reference_OTUs-Greengenes-illumina-16S-v4-5c6506,128222,82492,Qiita context
16,Pick_closed-reference_OTUs-Greengenes-illumina-16S-v4-90nt-44feac,125354,73083,Qiita context
19,Deblur-NA-illumina-16S-v4-100nt-fbc5b2,123127,5587560,Qiita context
90,Deblur-NA-illumina-16S-v4-90nt-99d1d8,119538,4460311,Qiita context


For simplicity sake, let's take the first context. It's large, and the phylogeny associated with the operational taxonomic units is easy to get. We'll break down the meaning of the context name in a moment. In practice, you will _most likely_ want to use the Deblur data, however producing a reasonable tree from those data requies a slightly computationally expensive step, and I'm on my laptop right now with limited battery quite literally in the middle of nowhere on a bus in the Czech Republic.

In [6]:
ctx = contexts.sort_values('SamplesWithData', ascending=False).iloc[0]['ContextName']

In [7]:
ctx

'Pick_closed-reference_OTUs-Greengenes-illumina-16S-v4-100nt-a243a1'

Breaking this name into its constiuent pieces, this is a closed reference context meaning that operational taxonomic units were assessed against a reference database and sequences which did not recruit to the reference were discarded. The reference used is Greengenes, a common 16S reference database. The gene represented by the data is the 16S SSU rRNA gene, and specifically the V4 region of the gene. Finally, the fragments represented are truncated to 100 nucleotides. (Don't worry if this is all a lot of jargon. It is a lot of jargon. Please ask questions :)

So cool, we have a "context". What can we do now? Let's search for some sample identifiers based off of the metadata (i.e., variables) associated with the samples. Specifically, let's get some skin, oral and fecal samples. Be forewarned, the metadata search uses Python's `ast` module behind the scenes, so malformed queries at present produce tracebacks.

In [8]:
study_id = 10317  # the Qiita study ID of the American Gut Project is 10317

query = "where qiita_study_id==%d" % (study_id)
results = redbiom.search.metadata_full(query)

In [9]:
len(results)

21506

In [32]:
results.keys()

dict_keys(['saliva', 'sebum', 'feces'])

In [33]:
study_id = 10317  # the Qiita study ID of the American Gut Project is 10317
results = {}
for site in ['sebum', 'saliva', 'feces']:
    query = "where qiita_study_id==%d and env_material=='%s'" % (study_id, site)
    results[site] = redbiom.search.metadata_full(query)

In [34]:
for k, v in results.items():
    print(k, len(v))

saliva 1257
sebum 1136
feces 16207


## Insert Ryan - Want to get metadata for all samples and export

In [38]:
to_keep_all = []
for k, v in results.items():
    to_keep_all.extend(list(v))

In [39]:
to_keep_all
md_all, _ = redbiom.fetch.sample_metadata(to_keep_all, context=ctx)

In [40]:
md_all.head(2)

,#SampleID,alcohol_types_beercider,alcohol_types_red_wine,alcohol_types_sour_beers,alcohol_types_spiritshard_alcohol,alcohol_types_unspecified,alcohol_types_white_wine,allergic_to_i_have_no_food_allergies_that_i_know_of,allergic_to_other,allergic_to_peanuts,...,subset_age,subset_antibiotic_history,subset_bmi,subset_diabetes,subset_healthy,subset_ibd,survey_id,taxon_id,title,weight_units
47968_10317.000001299,10317.000001299.47968,false,false,false,false,true,false,false,false,false,...,true,true,true,true,true,true,2a5fe0c2374071d6,408170,American Gut Project,kilograms
47968_10317.000001400,10317.000001400.47968,false,false,false,false,true,false,false,false,false,...,false,true,true,true,false,true,830667d87acfb31a,408170,American Gut Project,kilograms


In [42]:
print(md_all.shape)

(15364, 52)


As you can see, the sample sets are not very balanced. For the purposes of the mini-analysis, let's just grab the first 100 from each sample type.

In [12]:
to_keep = []
for k, v in results.items():
    to_keep.extend(list(v)[:100])

In [13]:
to_keep[:5]

['10317.000027335',
 '10317.000007428',
 '10317.000035471',
 '10317.000038436',
 '10317.000007312']

The last output cell shows what these IDs look like. These are Qiita sample IDs.

Now that we have some samples, let's get some data! What we're going to do is ask `redbiom` to obtain the sample data, for our `to_keep` samples, from the context we previously selected. What's happening behind the scenes is that the API is pulling out sparse vectors corresponding to the number of individual sequences observed for each operational taxonomic unit per sample, and additionally unmunging the names (as `redbiom` normalizes sample and feature identifiers). The output is then aggregated into what's called a BIOM `Table`, which is really just a rich object wrapped around a `scipy.sparse` matrix. 

You may noice two outputs on the return. The one we're ignoring represents "ambiguous" samples. Some sample identifiers are associated with multiple sequenced samples. This is because some samples may "fail" sequencing, where they didn't yield sufficient sequence data, and were rerun. These "failures" are still represented in Qiita, but are differentiated by the actual sequencing run they were on. This doesn't matter for the moment though.

In [14]:
biom_table, _ = redbiom.fetch.data_from_samples(ctx, to_keep)

In [15]:
biom_table

13713 x 234 <class 'biom.table.Table'> with 106154 nonzero entries (3% dense)

The `repr` output shows that we have roughly 13k OTUs (operational taxonomic units), and only 244 samples. What gives? We were supposed to get 300! Just because a sample has metadata does not mean it has sequence data. It is also possible that some of the samples haven't been run through the same bioinformatic processing (e.g., closed reference at 100nt).

More information on `biom` can be found [here](http://biom-format.org/). 

Let's play with the object for just a moment for familiarity.

In [16]:
print(biom_table.head(50)) # u

# Constructed from biom file
#OTU ID	10317.000043162.47998	10317.000013418.47970	10317.000036442.47991	10317.000033321.48017	10317.000069010.48072
4477479	0.0	0.0	0.0	0.0	0.0
190365	0.0	0.0	0.0	0.0	0.0
303448	0.0	0.0	0.0	0.0	0.0
92535	0.0	25.0	1.0	28.0	0.0
339015	0.0	0.0	0.0	0.0	0.0
332574	0.0	0.0	0.0	0.0	0.0
338553	0.0	0.0	0.0	0.0	0.0
4303475	0.0	0.0	0.0	0.0	0.0
513586	0.0	0.0	0.0	0.0	0.0
317385	0.0	0.0	0.0	0.0	0.0
4419621	0.0	0.0	0.0	0.0	0.0
4479286	0.0	0.0	0.0	0.0	0.0
3407050	0.0	0.0	0.0	0.0	0.0
113309	0.0	0.0	0.0	0.0	0.0
91721	0.0	0.0	0.0	0.0	0.0
4419004	0.0	0.0	0.0	0.0	0.0
748462	0.0	0.0	0.0	0.0	0.0
1870447	0.0	0.0	0.0	0.0	0.0
189548	0.0	0.0	0.0	0.0	0.0
811516	0.0	0.0	0.0	0.0	0.0
1136276	0.0	0.0	0.0	0.0	0.0
4437525	0.0	0.0	0.0	0.0	0.0
614380	0.0	0.0	0.0	0.0	0.0
338438	0.0	0.0	0.0	0.0	0.0
2867955	0.0	0.0	0.0	0.0	0.0
73559	0.0	0.0	0.0	0.0	0.0
1053979	0.0	0.0	0.0	0.0	0.0
2388088	0.0	0.0	0.0	0.0	0.0
1084983	0.0	0.0	0.0	0.0	0.0
2978122	1.0	0.0	0.0	0.0	0.0
749837	0.0	71.0	0.0	6.0	0.0
52

In [17]:
biom_table.head()

5 x 5 <class 'biom.table.Table'> with 3 nonzero entries (12% dense)

In [18]:
# lookup for ITU ID's and what they mean - linking them to higher level?

In [19]:
print(biom_table.head())

# Constructed from biom file
#OTU ID	10317.000043162.47998	10317.000013418.47970	10317.000036442.47991	10317.000033321.48017	10317.000069010.48072
4477479	0.0	0.0	0.0	0.0	0.0
190365	0.0	0.0	0.0	0.0	0.0
303448	0.0	0.0	0.0	0.0	0.0
92535	0.0	25.0	1.0	28.0	0.0
339015	0.0	0.0	0.0	0.0	0.0


In [20]:
df_biom = biom_table.to_dataframe()

In [21]:
df_biom.shape

(13713, 234)

In [22]:
df_biom.head()

,10317.000043162.47998,10317.000013418.47970,10317.000036442.47991,10317.000033321.48017,10317.000069010.48072,10317.000001590.47990,10317.000069158.48069,10317.000003174.47986,10317.000014523.48033,10317.000015151.47982,...,10317.000017778.48104,10317.000006974.47989,10317.000047757.48066,10317.000021819.48031,10317.000023383.48047,10317.000033374.47998,10317.000016447.47970,10317.000063464.48105,10317.000002253.47986,10317.000016359.47982
4477479,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
190365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
303448,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
92535,0.0,25.0,1.0,28.0,0.0,0.0,12.0,2.0,0.0,0.0,...,6.0,25.0,39.0,0.0,0.0,0.0,6.0,0.0,0.0,1.0
339015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


When we print the table, it implicitly casts it to a dense matrix representation. One thing to note: the sample IDs look different than before, right? The way that the individual sequencing runs are denoted is the last number -- the sample ID is represented by "qiita_study_id.the_actual_id.the_sequencing_run_id". This is to ensure the identifier is "globally" unique.

The identifiers on the rows are the "Greengenes" IDs. Also called OTU IDs. Or Feature IDs.

So we have a table. Now let's get the sample metadata. This will come down as a pandas `DataFrame`.

In [23]:
md, _ = redbiom.fetch.sample_metadata(to_keep, context=ctx)

In [24]:
md.head()

,#SampleID,acne_medication,acne_medication_otc,age_cat,age_corrected,age_years,alcohol_consumption,alcohol_types_beercider,alcohol_types_red_wine,alcohol_types_sour_beers,...,subset_healthy,subset_ibd,survey_id,taxon_id,thyroid,title,tonsils_removed,types_of_plants,weight_kg,weight_units
47969_10317.000004764,10317.000004764.47969,false,false,30s,36.0,36.0,true,false,false,false,...,true,true,63432d56c9957700,539655,Not provided,American Gut Project,Not provided,6 to 10,76.0,kilograms
47969_10317.000010977,10317.000010977.47969,false,false,60s,60.0,60.0,true,false,false,false,...,true,true,1c3a0c28db095253,447426,Not provided,American Gut Project,true,11 to 20,56.0,kilograms
47970_10317.000007780,10317.000007780.47970,false,true,30s,38.0,38.0,true,false,false,false,...,true,true,3409f245b0d580bd,539655,Not provided,American Gut Project,true,6 to 10,72.0,kilograms
47970_10317.000007785,10317.000007785.47970,false,false,child,9.0,9.0,false,false,false,false,...,false,true,63858372b3359691,539655,Not provided,American Gut Project,false,6 to 10,18.0,kilograms
47970_10317.000007797,10317.000007797.47970,false,false,child,5.0,5.0,false,false,false,false,...,false,true,338e3bbc86c88f5f,539655,Not provided,American Gut Project,false,6 to 10,15.0,kilograms


In [25]:
md.columns

Index(['#SampleID', 'acne_medication', 'acne_medication_otc', 'age_cat',
       'age_corrected', 'age_years', 'alcohol_consumption',
       'alcohol_types_beercider', 'alcohol_types_red_wine',
       'alcohol_types_sour_beers',
       ...
       'subset_healthy', 'subset_ibd', 'survey_id', 'taxon_id', 'thyroid',
       'title', 'tonsils_removed', 'types_of_plants', 'weight_kg',
       'weight_units'],
      dtype='object', length=130)

In [26]:
md.iloc[0]

#SampleID                                                                                   10317.000004764.47969
acne_medication                                                                                             false
acne_medication_otc                                                                                         false
age_cat                                                                                                       30s
age_corrected                                                                                                36.0
age_years                                                                                                    36.0
alcohol_consumption                                                                                          true
alcohol_types_beercider                                                                                     false
alcohol_types_red_wine                                                                  

In [27]:
md.sample_type.value_counts()

Stool         78
Mouth         73
Right Hand    31
Forehead      28
Left Hand     12
Torso          6
Left leg       5
Hair           1
Name: sample_type, dtype: int64

Great! So we have a bunch of sample data, and some study variables! Now for the QIIMEing. What we'll need to do is pacakge these data into types that QIIME 2 understands. And in particular, we need to use the semantic type system. This is well documented on the QIIME 2 website, and I recommend reviewing there.

In [44]:
table_ar = qiime2.Artifact.import_data('FeatureTable[Frequency]', biom_table)
md['age_cat'] = md['age_cat'].str.strip()
md['drinks_per_session'] = md['drinks_per_session'].str.strip()
md_ar = qiime2.Metadata(md.set_index('#SampleID'))

Ah! We need a tree! Since we're using Greengenes, we can just rely on the existing prebuilt tree from the reference. Let's get that.

In [45]:
!wget ftp://ftp.microbio.me/greengenes_release/gg_13_8_otus/trees/97_otus.tree

--2019-01-24 22:06:39--  ftp://ftp.microbio.me/greengenes_release/gg_13_8_otus/trees/97_otus.tree
           => '97_otus.tree.7'
Resolving ftp.microbio.me... 169.228.46.98
Connecting to ftp.microbio.me|169.228.46.98|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /greengenes_release/gg_13_8_otus/trees ... done.
==> SIZE 97_otus.tree ... 2542738
==> PASV ... done.    ==> RETR 97_otus.tree ... done.
Length: 2542738 (2.4M) (unauthoritative)

97_otus.tree.7      100%[===================>]   2.42M  1.82MB/s    in 1.3s    

2019-01-24 22:06:44 (1.82 MB/s) - '97_otus.tree.7' saved [2542738]



In [46]:
tree_ar = qiime2.Artifact.import_data('Phylogeny[Rooted]', '97_otus.tree')

Excellent. So next up, we need to rarefy these data. Rarefaction is a just randomly subsampling the samples without replacement in order to normalize for sequencing effort. There are a lot of ways to do this type of normalization. Rarefaction is dirty as you through out data, but it's pragmatic and tends to work pretty well. The issue rarefaction helps to ameliorate is that, the more you sequence the more life you'll observe. So if you don't do something to normalize sequencing effort across samples, you'll have a bad time.

Let's import all the plugins we'll be using.

In [47]:
from qiime2.plugins import feature_table, diversity, emperor

In [48]:
# rarefy to 1000 sequences per sample (yes, it's arbitrary)
rare_ar, = feature_table.actions.rarefy(table=table_ar, sampling_depth=1000)

In [49]:
unweighted_unifrac_ar, = diversity.actions.beta_phylogenetic(table=rare_ar, 
                                                             phylogeny=tree_ar, 
                                                             metric='unweighted_unifrac')

In [50]:
rare_ar

<artifact: FeatureTable[Frequency] uuid: 7a9bd9c6-8ed2-4261-bb13-80ca652a6cf8>

So before we move on, we should discuss what this voodoo just did. Beta diversity is what ecologists call comparing how similar (or dissimilar) two samples are. For example, how similar is a forest in the Pacific Northwest to a forest in Maine? (i.e., beta diversity is a function which takes two vectors and returns a scaler). We perform this calculation over all pairs of samples though so that we can examine all of the sample relationships. This distance matrix gets large quick: in our case, it's already 244 by 244. 

Visualizing that matrix would suck. So, one thing we often do is principal *coordinates* analysis. It's very similar to principal *components* analysis, except that we can pass in our distance matrix.

In [51]:
unweighted_unifrac_pcoa_ar, = diversity.actions.pcoa(unweighted_unifrac_ar)

//anaconda/envs/qiime2-2018.11/lib/python3.5/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:152: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.020043480978383574 and the largest is 13.74605277592779.
  RuntimeWarning


Finally, let's actually view the coordinates. It's much more interesting to use the metadata as well :) Hint: color by body site.

In [52]:
viz, = emperor.actions.plot(unweighted_unifrac_pcoa_ar, md_ar)

In [53]:
viz

<visualization: Visualization uuid: f3ef5c38-d0c8-4b07-b6b7-8864e16ea6f6>